In [25]:
import numpy as np
import pandas as pd

In [38]:
# 数据准备，为了让优化的效果表现的明显，维度用的是十万
x = np.random.randint(10, size=(100000,1))
y = np.random.randint(2, size=(100000,1))
data = pd.DataFrame(np.concatenate([y,x],axis=1),columns=['y','x'])
x1 = x.ravel()
y1 = y.ravel()

In [27]:
def target_mean_v1(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
    groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

In [28]:
def target_mean_v2(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data.loc[i, x_name] not in value_dict.keys():
      value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] = 1
    else:
      value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] += 1
  for i in range(data.shape[0]):
      result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
  return result

In [29]:
%%timeit
target_mean_v2(data,'y','x')

1 loop, best of 3: 5 s per loop


pd.DataFrame的检索是代码瓶颈所在，通过实验，发现本质上传入的参数是两个np.ndarray,因此通过改变传入参数的方法优化得到target_mean_v3。

In [46]:
def target_mean_v3(x, y):
  n = x.shape[0]
  result = np.empty(n)
  value_dict = {}
  count_dict = {}
  for i in range(n):
    xi = x[i]
    yi = y[i]
    if x[i] not in value_dict:
      value_dict[xi] = yi
      count_dict[xi] = 1
    else:
      value_dict[xi] += yi
      count_dict[xi] += 1
  for i in range(n):
    xi = x[i]
    yi = y[i]
    result[i] = (value_dict[xi] - yi) / (count_dict[xi] - 1)
  return result

In [47]:
%%timeit
target_mean_v3(x1,y1)

10 loops, best of 3: 169 ms per loop


代码中的if...else...作用是判断x[i]是否在字典中，因此考虑通过数组下标达到类似于字典的key来去掉if...else...,并且提前传入x数组中数据的最大值+1作为要创建的数组大小。

In [52]:
def target_mean_v4(n, x, y):
  values_dict = np.zeros(n, dtype=np.intc)
  count_dict = np.zeros(n, dtype=np.intc)
  nrow = x.shape[0]
  result = np.empty(nrow,dtype=np.float64)
  for i in range(nrow):
    xi = x[i]
    yi = y[i]
    values_dict[xi] += yi
    count_dict[xi] += 1
  for i in range(nrow):
    xi = x[i]
    yi = y[i]
    result[xi] = (values_dict[xi] - yi) / (count_dict[xi] - 1)
  return result

到这里v4还没经过测试，但推测v4和下一版v5可能没有v3效率高，因为字典是使用hash查找的，查找效率应该优于数组？另：v5优化的地方是：用到zip函数将nrow去掉。

In [53]:
def target_mean_v5(n, x, y):
  values_dict = np.zeros(n, dtype=np.intc)
  count_dict = np.zeros(n, dtype=np.intc)
  for i, j in zip(x,y):
    values_dict[i] += j
    count_dict[i] += 1
  result = [(values_dict[i] - j) / (count_dict[i] - 1) for i, j in zip(x,y)]
  result = np.array(result)
  return result

速度确实比原来慢了，需要重新考虑values_dict和count_dict的数据结构。

In [67]:
%%timeit 
target_mean_v4(10,x1,y1)

1 loop, best of 3: 976 ms per loop


zip的优化基本没效果

In [57]:
%%timeit 
target_mean_v5(10,x1,y1)

1 loop, best of 3: 978 ms per loop


通过查阅资料，找到python标准库collections中有defaultdict这种数据结构，可以去掉if...else...循环



In [70]:
from collections import defaultdict
def target_mean_v6(x, y):
  values_dict = defaultdict(lambda:0)
  count_dict = defaultdict(lambda:0)
  n = x.shape[0]
  result = np.empty(n,dtype=np.float64)
  for i in range(n):
    xi = x[i]
    yi = y[i]
    values_dict[xi] += yi
    count_dict[xi] += 1
  for i in range(n):
    xi = x[i]
    yi = y[i]
    result[i] = (values_dict[xi] - yi) / (count_dict[xi] - 1)
  return result

测试过几次，比v3版本要快，但c/c++中没有类似的数据结构，因此决定以v3版本作为修改模板来完成cython的实现。

In [72]:
%%timeit
target_mean_v6(x1,y1)

10 loops, best of 3: 154 ms per loop


In [ ]:
%load_ext Cython

字典功能用的是c++的（unordered_map或者map）？来实现。



In [73]:
%%cython --cplus
import numpy as np
cimport numpy as cnp
cimport cython
from libcpp.map cimport map

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v7(long[:] x, long[:] y):
  cdef:
    int n = x.shape[0]
    int i
    long xi,yi
    double[:] result = np.zeros(n)
    map[long,long] value_dict, count_dict
  for i in range(n):
    xi = x[i]
    yi = y[i]
    if value_dict.count(x[i]):
      value_dict[xi] += yi
      count_dict[xi] += 1
    else:
      value_dict[xi] = yi
      count_dict[xi] = 1
  for i in range(n):
    xi = x[i]
    yi = y[i]
    result[i] = (value_dict[xi]-yi) / (count_dict[xi]-1)
  return result

这里测试map优于unordered_map

In [74]:
%%timeit
target_mean_v7(x1,y1)

100 loops, best of 3: 7.28 ms per loop


尝试使用数组。

In [75]:
%%cython --cplus
import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double[:] target_mean_v8(int n, long[:] x, long[:] y):
  cdef:
    int nrow = x.shape[0]
    int i
    long xi,yi
    double[:] result = np.empty(nrow)
    long[:] value_dict = np.zeros(n, dtype=np.int64), count_dict = np.zeros(n,dtype=np.int64)
  for i in range(nrow):
    xi = x[i]
    yi = y[i]
    value_dict[xi] += yi
    count_dict[xi] += 1
  for i in range(nrow):
    xi = x[i]
    yi = y[i]
    result[i] = (value_dict[xi]-yi) / (count_dict[xi]-1)
  return result

In [76]:
%%timeit
target_mean_v8(10,x1,y1)

1000 loops, best of 3: 446 µs per loop


In [77]:
%%cython --cplus
import numpy as np
cimport numpy as cnp
cimport cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double[:] target_mean_v9(const int n, long[:] x, long[:] y):
  cdef:
    int nrow = x.shape[0]
    int i
    double[:] result = np.asfortranarray(np.empty(nrow))
    long xi,yi
    long[:] value_dict = np.zeros(n,dtype=np.int64),count_dict = np.zeros(n,dtype=np.int64)
  for i in prange(nrow,nogil=True,num_threads=8):
    xi = x[i]
    yi = y[i]
    value_dict[xi] += yi
    count_dict[xi] += 1
  for i in prange(nrow,nogil=True,num_threads=8):
    xi = x[i]
    yi = y[i]
    result[i] = (value_dict[xi]-yi) / (count_dict[xi]-1)
  return result

多线程实现的v9比原来的v8要慢

In [78]:
%%timeit
target_mean_v9(10,x1,y1)

The slowest run took 4.63 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 508 µs per loop
